In [1]:
# 1단계: MariaDB 연결 및 기본 설정
!pip install mysql-connector-python

In [37]:
import mysql.connector

# 데이터베이스 연결 설정
db_config = {
    'host': '*********',  # NAS IP 주소
    'user': 'junseo',        # MariaDB 사용자 이름
    'password': '*********',  # MariaDB 비밀번호
    'database': 'PronounceDB'     # 데이터베이스 이름
}

# 데이터베이스 연결 테스트
try:
    conn = mysql.connector.connect(**db_config)  # MariaDB 연결
    print("MariaDB 연결 성공!")
except mysql.connector.Error as err:
    print(f"MariaDB 연결 실패: {err}")
    conn = None

MariaDB 연결 성공!


In [4]:
import os
from tqdm import tqdm

# NAS 공유 폴더 경로
nas_folder_path = "/Volumes/준서/ML용 데이터/037.교육용 한국인의 영어 음성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TS_PRN_en_NA_NA"

# 모든 하위 디렉토리의 JSON 파일 찾기
def find_all_json_files(root_folder):
    json_files = []
    total_files = sum(len(files) for _, _, files in os.walk(root_folder))  # 전체 파일 개수 계산
    progress_bar = tqdm(total=total_files, desc="JSON 파일 탐색 중")  # 프로그레스 바 생성

    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            progress_bar.update(1)  # 파일 탐색 진행 상태 업데이트
            if file.endswith(".json"):
                json_files.append(os.path.join(dirpath, file))

    progress_bar.close()
    print(f"탐색 완료! 총 {len(json_files)}개의 JSON 파일 발견")
    return json_files

# JSON 파일 탐색 실행
json_files = find_all_json_files(nas_folder_path)

JSON 파일 탐색 중: 100%|██████████| 21716/21716 [00:46<00:00, 470.84it/s]

탐색 완료! 총 21716개의 JSON 파일 발견


In [34]:
# 테이블 생성 쿼리 목록
TABLE_CREATION_QUERIES = [
    ("MetaData", """
    CREATE TABLE IF NOT EXISTS MetaData (
        id INT AUTO_INCREMENT PRIMARY KEY,
        recrdDt DATE NOT NULL,
        recrdTime FLOAT NOT NULL,
        fileNm VARCHAR(255) NOT NULL,
        recrdPlace VARCHAR(50),
        recrdDevice VARCHAR(50),
        peakLevel FLOAT,
        rmsLevel FLOAT
    );
    """),
    ("Recorder", """
    CREATE TABLE IF NOT EXISTS Recorder (
        id INT AUTO_INCREMENT PRIMARY KEY,
        recorderId INT NOT NULL,
        motherLang VARCHAR(50),
        gender CHAR(1),
        age INT,
        ability VARCHAR(50),
        metaDataId INT,
        FOREIGN KEY (metaDataId) REFERENCES MetaData(id)
    );
    """),
    ("ScriptInfo", """
    CREATE TABLE IF NOT EXISTS ScriptInfo (
        id INT AUTO_INCREMENT PRIMARY KEY,
        scriptName VARCHAR(100),
        scriptId VARCHAR(100),
        language VARCHAR(50),
        evaluationArea VARCHAR(50),
        metaDataId INT,
        FOREIGN KEY (metaDataId) REFERENCES MetaData(id)
    );
    """),
    ("ItemScript", """
    CREATE TABLE IF NOT EXISTS ItemScript (
        id INT AUTO_INCREMENT PRIMARY KEY,
        sentence TEXT,
        sttText TEXT,
        transAnsw TEXT,
        phonemeText TEXT,
        tagging TEXT,
        scriptInfoId INT,
        FOREIGN KEY (scriptInfoId) REFERENCES ScriptInfo(id)
    );
    """),
    ("PronFeatures", """
    CREATE TABLE IF NOT EXISTS PronFeatures (
        id INT AUTO_INCREMENT PRIMARY KEY,
        ros FLOAT,
        ar FLOAT,
        ptr FLOAT,
        wpsec FLOAT,
        wpsecutt FLOAT,
        silpsec FLOAT,
        silmean FLOAT,
        silmeandev FLOAT,
        longpfreq FLOAT,
        longpmn FLOAT,
        longpwd FLOAT,
        longpmeandev FLOAT,
        silstddev FLOAT,
        longpstdev FLOAT,
        metaDataId INT,
        FOREIGN KEY (metaDataId) REFERENCES MetaData(id)
    );
    """
    )
]

# 테이블 생성 함수
def create_tables():
    try:
        # 데이터베이스 연결
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # 각 테이블 생성
        for table_name, query in TABLE_CREATION_QUERIES:
            cursor.execute(query)
            print(f"테이블 '{table_name}' 생성 완료")

        conn.commit()
        print("모든 테이블 생성 성공!")

    except mysql.connector.Error as err:
        print(f"에러 발생: {err}")

    finally:
        # 연결 닫기
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print("MariaDB 연결 종료")

# 실행
if __name__ == "__main__":
    create_tables()

테이블 'MetaData' 생성 완료
테이블 'Recorder' 생성 완료
테이블 'ScriptInfo' 생성 완료
테이블 'ItemScript' 생성 완료
테이블 'PronFeatures' 생성 완료
모든 테이블 생성 성공!
MariaDB 연결 종료


In [35]:
def insert_data_to_db(json_data, conn):
    try:
        cursor = conn.cursor()

        # MetaData 삽입
        meta_query = """
        INSERT INTO MetaData (recrdDt, recrdTime, fileNm, recrdPlace, recrdDevice, peakLevel, rmsLevel)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        meta_values = (
            json_data.get('recrdDt'),
            json_data.get('recrdTime'),
            json_data.get('fileNm'),
            json_data.get('recrdPlace'),
            json_data.get('recrdDevice'),
            json_data.get('peakLevel'),
            json_data.get('rmsLevel')
        )
        cursor.execute(meta_query, meta_values)
        metaDataId = cursor.lastrowid  # MetaData ID 가져오기

        # Recorder 삽입
        recorder = json_data.get('recorder', {})
        recorder_query = """
        INSERT INTO Recorder (recorderId, motherLang, gender, age, ability, metaDataId)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        recorder_values = (
            recorder.get('recorderId'),
            recorder.get('motherLang'),
            recorder.get('gender'),
            recorder.get('age'),
            recorder.get('ability'),
            metaDataId
        )
        cursor.execute(recorder_query, recorder_values)

        # ScriptInfo 삽입
        script_info = json_data.get('scriptInfo', {})
        script_query = """
        INSERT INTO ScriptInfo (scriptName, scriptId, language, evaluationArea, metaDataId)
        VALUES (%s, %s, %s, %s, %s)
        """
        script_values = (
            script_info.get('scriptName'),
            script_info.get('scriptId'),
            script_info.get('language'),
            script_info.get('evaluationArea'),
            metaDataId
        )
        cursor.execute(script_query, script_values)
        scriptInfoId = cursor.lastrowid  # ScriptInfo ID 가져오기

        # ItemScript 삽입
        item_script = json_data.get('itemScript', {}).get('contents', [])
        item_query = """
        INSERT INTO ItemScript (sentence, sttText, transAnsw, phonemeText, tagging, scriptInfoId)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        for item in item_script:
            item_values = (
                item.get('sentence'),
                item.get('sttText'),
                item.get('transAnsw'),
                item.get('phonemeText'),
                item.get('tagging'),
                scriptInfoId
            )
            cursor.execute(item_query, item_values)

        # PronFeatures 삽입
        pron_feat = json_data.get('pronFeat', {})
        pron_query = """
        INSERT INTO PronFeatures (ros, ar, ptr, wpsec, wpsecutt, silpsec, silmean, silmeandev, longpfreq, longpmn, longpwd, longpmeandev, silstddev, longpstdev, metaDataId)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        pron_values = (
            pron_feat.get('ros'),
            pron_feat.get('ar'),
            pron_feat.get('ptr'),
            pron_feat.get('wpsec'),
            pron_feat.get('wpsecutt'),
            pron_feat.get('silpsec'),
            pron_feat.get('silmean'),
            pron_feat.get('silmeandev'),
            pron_feat.get('longpfreq'),
            pron_feat.get('longpmn'),
            pron_feat.get('longpwd'),
            pron_feat.get('longpmeandev'),
            pron_feat.get('silstddev'),
            pron_feat.get('longpstdev'),
            metaDataId
        )
        cursor.execute(pron_query, pron_values)

        # 트랜잭션 커밋
        conn.commit()

    except mysql.connector.Error as e:
        print(f"SQL 실행 중 에러 발생: {e}")
        conn.rollback()
    finally:
        cursor.close()


In [25]:
import json

In [36]:
def process_json_files(json_files, conn):
    # tqdm 프로그레스바 생성
    with tqdm(total=len(json_files), desc="JSON 데이터 삽입 중", unit="file") as progress_bar:
        for file in json_files:
            try:
                with open(file, 'r', encoding='utf-8-sig') as f:
                    json_data = json.load(f)
                    insert_data_to_db(json_data, conn)  # 데이터 삽입 함수 호출
            except Exception as e:
                print(f"{file} 처리 중 에러 발생: {e}")
            finally:
                progress_bar.update(1)  # 프로그레스바 업데이트

# 실행
try:
    conn = mysql.connector.connect(**db_config)
    process_json_files(json_files, conn)
finally:
    if conn.is_connected():
        conn.close()

JSON 데이터 삽입 중: 100%|██████████| 21716/21716 [4:30:14<00:00,  1.34file/s]   IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)


